In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import collections
import argparse
import sys
import pickle
import numpy as np
import time, datetime
import copy
from thop import profile
from collections import OrderedDict
import shutil
import torch.utils
import torch.utils.data.distributed
from torchvision import datasets, transforms
import json
from scipy.spatial import distance

In [2]:
print('==> Building mobileNetV2 model..')
import math
import pdb
def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )
def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )
def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)
class MobileNetV2(nn.Module):
    def __init__(self, compress_rate, n_class, input_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        interverted_residual_setting = [
            # t-ex, c-channel, n-blocknum, s-stride
            [1, 16, 1, 1],
            [6, 24, 2, 1], # NOTE: change stride 2 -> 1 for CIFAR10
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]
        self.compress_rate=compress_rate[:]

        # building first layer
        assert input_size % 32 == 0
        # input_channel = make_divisible(input_channel * width_mult)  # first channel is always 32!
        self.last_channel = make_divisible(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2)]
        # building inverted residual blocks
        cnt=1
        for t, c, n, s in interverted_residual_setting:
            output_channel = make_divisible(c * width_mult) if t > 1 else c
            output_channel = int((1-self.compress_rate[cnt])*output_channel)
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, expand_ratio=t))
                else:
                    self.features.append(block(input_channel, output_channel, 1, expand_ratio=t))
                input_channel = output_channel
            cnt+=1

        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        #self.classifier = nn.Linear(self.last_channel, n_class)
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, n_class),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

def mobilenet_v2(compress_rate,n_class=9):
    model = MobileNetV2(compress_rate=compress_rate,n_class=n_class,width_mult=1)
    return model

==> Building mobileNetV2 model..


In [3]:
print("prepare resNet_50 model...")
def adapt_channel(compress_rate, num_layers):

    if num_layers==56:
        stage_repeat = [9, 9, 9]
        stage_out_channel = [16] + [16] * 9 + [32] * 9 + [64] * 9
    elif num_layers==110:
        stage_repeat = [18, 18, 18]
        stage_out_channel = [16] + [16] * 18 + [32] * 18 + [64] * 18

    stage_oup_cprate = []
    stage_oup_cprate += [compress_rate[0]]
    for i in range(len(stage_repeat)-1):
        stage_oup_cprate += [compress_rate[i+1]] * stage_repeat[i]
    stage_oup_cprate +=[0.] * stage_repeat[-1]
    mid_cprate = compress_rate[len(stage_repeat):]

    overall_channel = []
    mid_channel = []
    for i in range(len(stage_out_channel)):
        if i == 0 :
            overall_channel += [int(stage_out_channel[i] * (1-stage_oup_cprate[i]))]
        else:
            overall_channel += [int(stage_out_channel[i] * (1-stage_oup_cprate[i]))]
            mid_channel += [int(stage_out_channel[i] * (1-mid_cprate[i-1]))]

    return overall_channel, mid_channel
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, midplanes, inplanes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.inplanes = inplanes
        self.planes = planes
        self.conv1 = conv3x3(inplanes, midplanes, stride)
        self.bn1 = nn.BatchNorm2d(midplanes)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(midplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.stride = stride

        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != planes:
            if stride!=1:
                self.shortcut = LambdaLayer(
                    lambda x: F.pad(x[:, :, ::2, ::2],
                                    (0, 0, 0, 0, (planes-inplanes)//2, planes-inplanes-(planes-inplanes)//2), "constant", 0))
            else:
                self.shortcut = LambdaLayer(
                    lambda x: F.pad(x[:, :, :, :],
                                    (0, 0, 0, 0, (planes-inplanes)//2, planes-inplanes-(planes-inplanes)//2), "constant", 0))
            #self.shortcut = LambdaLayer(
            #    lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4),"constant", 0))

            '''self.shortcut = nn.Sequential(
                conv1x1(inplanes, planes, stride=stride),
                #nn.BatchNorm2d(planes),
            )#'''

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        #print(self.stride, self.inplanes, self.planes, out.size(), self.shortcut(x).size())
        out += self.shortcut(x)
        out = self.relu2(out)

        return out
class ResNet(nn.Module):
    def __init__(self, block, num_layers, compress_rate, num_classes):
        super(ResNet, self).__init__()
        assert (num_layers - 2) % 6 == 0, 'depth should be 6n+2'
        n = (num_layers - 2) // 6

        self.num_layer = num_layers
        self.overall_channel, self.mid_channel = adapt_channel(compress_rate, num_layers)

        self.layer_num = 0
        self.conv1 = nn.Conv2d(3, self.overall_channel[self.layer_num], kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.overall_channel[self.layer_num])
        self.relu = nn.ReLU(inplace=True)
        self.layers = nn.ModuleList()
        self.layer_num += 1

        #self.layers = nn.ModuleList()
        self.layer1 = self._make_layer(block, blocks_num=n, stride=1)
        self.layer2 = self._make_layer(block, blocks_num=n, stride=2)
        self.layer3 = self._make_layer(block, blocks_num=n, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        if self.num_layer == 56:
            self.fc = nn.Linear(64 * BasicBlock.expansion, num_classes)
        else:
            self.linear = nn.Linear(64 * BasicBlock.expansion, num_classes)


    def _make_layer(self, block, blocks_num, stride):
        layers = []
        layers.append(block(self.mid_channel[self.layer_num - 1], self.overall_channel[self.layer_num - 1],
                                 self.overall_channel[self.layer_num], stride))
        self.layer_num += 1

        for i in range(1, blocks_num):
            layers.append(block(self.mid_channel[self.layer_num - 1], self.overall_channel[self.layer_num - 1],
                                     self.overall_channel[self.layer_num]))
            self.layer_num += 1

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        for i, block in enumerate(self.layer1):
            x = block(x)
        for i, block in enumerate(self.layer2):
            x = block(x)
        for i, block in enumerate(self.layer3):
            x = block(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        if self.num_layer == 56:
            x = self.fc(x)
        else:
            x = self.linear(x)

        return x

def resnet_56(compress_rate,num_classes):
    return ResNet(BasicBlock, 56, compress_rate,num_classes)

def resnet_110(compress_rate,num_classes):
    return ResNet(BasicBlock, 110, compress_rate,num_classes)

prepare resNet_50 model...


In [4]:
print("prepare vgg...")
defaultcfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 8192, 512]
relucfg = [2, 6, 9, 13, 16, 19, 23, 26, 29, 33, 36, 39]
class VGG(nn.Module):
    def __init__(self, compress_rate, cfg=None, num_classes=9):
        super(VGG, self).__init__()

        if cfg is None:
            cfg = defaultcfg
        self.relucfg = relucfg

        self.compress_rate = compress_rate[:]
        self.compress_rate.append(0.0)

        self.features = self._make_layers(cfg)
        self.classifier = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(cfg[-2], cfg[-1])),
            ('norm1', nn.BatchNorm1d(cfg[-1])),
            ('relu1', nn.ReLU(inplace=True)),
            ('linear2', nn.Linear(cfg[-1], num_classes)),
        ]))

    def _make_layers(self, cfg):

        layers = nn.Sequential()
        in_channels = 3
        cnt=0

        for i, x in enumerate(cfg):
            if x == 'M':
                layers.add_module('pool%d' % i, nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                x = int(x * (1-self.compress_rate[cnt]))

                cnt+=1
                conv2d = nn.Conv2d(in_channels, x, kernel_size=3, padding=1)
                layers.add_module('conv%d' % i, conv2d)
                layers.add_module('norm%d' % i, nn.BatchNorm2d(x))
                layers.add_module('relu%d' % i, nn.ReLU(inplace=True))
                in_channels = x

        return layers

    def forward(self, x):
        x = self.features(x)

        x = nn.AvgPool2d(2)(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x  
def vgg_16_bn(compress_rate):
    return VGG(compress_rate=compress_rate)

prepare vgg...


In [5]:
print("prepare densent....")
norm_mean, norm_var = 0.0, 1.0
cov_cfg=[(3*i+1) for i in range(12*3+2+1)]
class DenseBasicBlock(nn.Module):
    def __init__(self, inplanes, outplanes, dropRate=0):
        super(DenseBasicBlock, self).__init__()

        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(inplanes, outplanes, kernel_size=3,
                               padding=1, bias=False)

        self.dropRate = dropRate

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        if self.dropRate > 0:
            out = F.dropout(out, p=self.dropRate, training=self.training)

        out = torch.cat((x, out), 1)

        return out
class Transition(nn.Module):
    def __init__(self, inplanes, outplanes):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(inplanes, outplanes, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = F.avg_pool2d(out, 2)
        return out
class DenseNet(nn.Module):

    def __init__(self, compress_rate, depth=40, block=DenseBasicBlock,
        dropRate=0, num_classes=9, growthRate=12, compressionRate=1):
        super(DenseNet, self).__init__()
        self.compress_rate=compress_rate

        assert (depth - 4) % 3 == 0, 'depth should be 3n+4'
        n = (depth - 4) // 3 if 'DenseBasicBlock' in str(block) else (depth - 4) // 6

        transition = Transition

        self.covcfg=cov_cfg

        self.growthRate = growthRate
        self.dropRate = dropRate

        self.inplanes = growthRate * 2
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, padding=1,
                               bias=False)

        self.dense1 = self._make_denseblock(block, n, compress_rate[1:n+1])
        self.trans1 = self._make_transition(transition, compressionRate, compress_rate[n+1])
        self.dense2 = self._make_denseblock(block, n, compress_rate[n+2:2*n+2])
        self.trans2 = self._make_transition(transition, compressionRate, compress_rate[2*n+2])
        self.dense3 = self._make_denseblock(block, n, compress_rate[2*n+3:3*n+3])
        self.bn = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8)

        self.fc = nn.Linear(self.inplanes, num_classes)

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_denseblock(self, block, blocks, compress_rate):
        layers = []
        for i in range(blocks):
            layers.append(block(self.inplanes, outplanes=int(self.growthRate*(1-compress_rate[i])), dropRate=self.dropRate))
            self.inplanes += int(self.growthRate*(1-compress_rate[i]))

        return nn.Sequential(*layers)

    def _make_transition(self, transition, compressionRate, compress_rate):
        inplanes = self.inplanes
        outplanes = int(math.floor(self.inplanes*(1-compress_rate) // compressionRate))
        self.inplanes = outplanes
        return transition(inplanes, outplanes)

    def forward(self, x):
        x = self.conv1(x)

        x = self.dense1(x)
        x = self.trans1(x)
        x = self.dense2(x)
        x = self.trans2(x)
        x = self.dense3(x)
        x = self.bn(x)
        x = self.relu(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        x = self.fc(x)

        return x
    
def densenet_40(compress_rate):
    return DenseNet(compress_rate=compress_rate, depth=40, block=DenseBasicBlock)

prepare densent....


In [6]:
print("prepare googlenet...")
'''GoogLeNet with PyTorch.'''
norm_mean, norm_var = 0.0, 1.0
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes,tmp_name, cp_rate, last=False):
        super(Inception, self).__init__()
        self.tmp_name=tmp_name

        self.n1x1 = n1x1
        self.n3x3 = n3x3
        self.n5x5 = n5x5
        self.pool_planes = pool_planes

        # 1x1 conv branch
        if self.n1x1:
            conv1x1 = nn.Conv2d(in_planes, n1x1, kernel_size=1)
            conv1x1.tmp_name = self.tmp_name

            self.branch1x1 = nn.Sequential(
                conv1x1,
                nn.BatchNorm2d(n1x1),
                nn.ReLU(True),
            )

        # 1x1 conv -> 3x3 conv branch
        if self.n3x3:

            if last:
                output=n3x3
            else:
                output=int(n3x3*cp_rate)

            conv3x3_1=nn.Conv2d(in_planes, n3x3red, kernel_size=1)
            conv3x3_2=nn.Conv2d(n3x3red, output, kernel_size=3, padding=1)
            conv3x3_1.tmp_name = self.tmp_name
            conv3x3_2.tmp_name = self.tmp_name

            self.branch3x3 = nn.Sequential(
                conv3x3_1,
                nn.BatchNorm2d(n3x3red),
                nn.ReLU(True),
                conv3x3_2,
                nn.BatchNorm2d(output),
                nn.ReLU(True),
            )

        # 1x1 conv -> 5x5 conv branch
        if self.n5x5 > 0:

            if last:
                output=n5x5
            else:
                output=int(n5x5*cp_rate)

            conv5x5_1 = nn.Conv2d(in_planes, n5x5red, kernel_size=1)
            conv5x5_2 = nn.Conv2d(n5x5red, int(n5x5*cp_rate), kernel_size=3, padding=1)
            conv5x5_3 = nn.Conv2d(int(n5x5*cp_rate), output, kernel_size=3, padding=1)
            conv5x5_1.tmp_name = self.tmp_name
            conv5x5_2.tmp_name = self.tmp_name
            conv5x5_3.tmp_name = self.tmp_name

            self.branch5x5 = nn.Sequential(
                conv5x5_1,
                nn.BatchNorm2d(n5x5red),
                nn.ReLU(True),
                conv5x5_2,
                nn.BatchNorm2d(int(n5x5*cp_rate)),
                nn.ReLU(True),
                conv5x5_3,
                nn.BatchNorm2d(output),
                nn.ReLU(True),
            )

        # 3x3 pool -> 1x1 conv branch
        if self.pool_planes > 0:
            conv_pool = nn.Conv2d(in_planes, pool_planes, kernel_size=1)
            conv_pool.tmp_name = self.tmp_name

            self.branch_pool = nn.Sequential(
                nn.MaxPool2d(3, stride=1, padding=1),
                conv_pool,
                nn.BatchNorm2d(pool_planes),
                nn.ReLU(True),
            )

    def forward(self, x):
        out = []
        y1 = self.branch1x1(x)
        out.append(y1)

        y2 = self.branch3x3(x)
        out.append(y2)

        y3 = self.branch5x5(x)
        out.append(y3)

        y4 = self.branch_pool(x)
        out.append(y4)
        return torch.cat(out, 1)
class GoogLeNet(nn.Module):
    def __init__(self, compress_rate, block=Inception, filters=None):
        super(GoogLeNet, self).__init__()

        first_outplanes=192
        conv_pre = nn.Conv2d(3, first_outplanes, kernel_size=3, padding=1)
        conv_pre.tmp_name = 'pre_layer'
        self.pre_layers = nn.Sequential(
            conv_pre,
            nn.BatchNorm2d(first_outplanes),
            nn.ReLU(True),
        )

        filters = [
            [64, 128, 32, 32],
            [128, 192, 96, 64],
            [192, 208, 48, 64],
            [160, 224, 64, 64],
            [128, 256, 64, 64],
            [112, 288, 64, 64],
            [256, 320, 128, 128],
            [256, 320, 128, 128],
            [384, 384, 128, 128]
        ]
        self.filters = filters

        mid_filters = [
            [96, 16],
            [128, 32],
            [96, 16],
            [112, 24],
            [128, 24],
            [144, 32],
            [160, 32],
            [160, 32],
            [192, 48]
        ]

        cp_rate_list=[]
        for cp_rate in compress_rate:
            cp_rate_list.append(1-cp_rate)

        in_plane_list=[]
        for i in range(8):
            in_plane_list.append(filters[i][0]+int(filters[i][1]*cp_rate_list[i+1])+int(filters[i][2]*cp_rate_list[i+1])+filters[i][3])

        self.inception_a3 = block(first_outplanes, filters[0][0], mid_filters[0][0], filters[0][1], mid_filters[0][1], filters[0][2], filters[0][3], 'a3', cp_rate_list[1])
        self.inception_b3 = block(in_plane_list[0], filters[1][0], mid_filters[1][0], filters[1][1], mid_filters[1][1], filters[1][2], filters[1][3], 'a4', cp_rate_list[2])

        self.maxpool1 = nn.MaxPool2d(3, stride=2, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, padding=1)

        self.inception_a4 = block(in_plane_list[1], filters[2][0], mid_filters[2][0], filters[2][1], mid_filters[2][1], filters[2][2], filters[2][3], 'a4', cp_rate_list[3])
        self.inception_b4 = block(in_plane_list[2], filters[3][0], mid_filters[3][0], filters[3][1], mid_filters[3][1], filters[3][2], filters[3][3], 'b4', cp_rate_list[4])
        self.inception_c4 = block(in_plane_list[3], filters[4][0], mid_filters[4][0], filters[4][1], mid_filters[4][1], filters[4][2], filters[4][3], 'c4', cp_rate_list[5])
        self.inception_d4 = block(in_plane_list[4], filters[5][0], mid_filters[5][0], filters[5][1], mid_filters[5][1], filters[5][2], filters[5][3], 'd4', cp_rate_list[6])
        self.inception_e4 = block(in_plane_list[5], filters[6][0], mid_filters[6][0], filters[6][1], mid_filters[6][1], filters[6][2], filters[6][3], 'e4', cp_rate_list[7])

        self.inception_a5 = block(in_plane_list[6], filters[7][0], mid_filters[7][0], filters[7][1], mid_filters[7][1], filters[7][2], filters[7][3], 'a5', cp_rate_list[8])
        self.inception_b5 = block(in_plane_list[7], filters[8][0], mid_filters[8][0], filters[8][1], mid_filters[8][1], filters[8][2], filters[8][3], 'b5', cp_rate_list[9], last=True)

        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(sum(filters[-1]), 9)

    def forward(self, x):

        out = self.pre_layers(x)

        # 192 x 32 x 32
        out = self.inception_a3(out)
        # 256 x 32 x 32
        out = self.inception_b3(out)
        # 480 x 32 x 32
        out = self.maxpool1(out)

        # 480 x 16 x 16
        out = self.inception_a4(out)
        # 512 x 16 x 16
        out = self.inception_b4(out)
        # 512 x 16 x 16
        out = self.inception_c4(out)
        # 512 x 16 x 16
        out = self.inception_d4(out)
        # 528 x 16 x 16
        out = self.inception_e4(out)
        # 823 x 16 x 16
        out = self.maxpool2(out)

        # 823 x 8 x 8
        out = self.inception_a5(out)
        # 823 x 8 x 8
        out = self.inception_b5(out)

        # 1024 x 8 x 8
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out
    
def googlenet(compress_rate):
    return GoogLeNet(compress_rate=compress_rate, block=Inception)

prepare googlenet...


In [7]:
# utils
def load_mobilenetv2_model(model, oristate_dict):

    state_dict = model.state_dict()

    last_select_index = None

    all_honey_conv_weight = []

    bn_part_name=['.weight','.bias','.running_mean','.running_var']
    prefix = rank_conv_prefix+'rank_conv'
    subfix = ".npy"

    layer_cnt=1
    conv_cnt=1
    cfg=[1,2,3,4,3,3,1,1]
    for layer, num in enumerate(cfg):
        if layer_cnt==1:
            conv_id=[0,3]
        elif layer_cnt==18:
            conv_id=[0]
        else:
            conv_id=[0,3,6]

        for k in range(num):
            if layer_cnt==18:
                block_name = 'features.' + str(layer_cnt) + '.'
            else:
                block_name = 'features.'+str(layer_cnt)+'.conv.'

            for l in conv_id:
                conv_cnt += 1
                conv_name = block_name + str(l)
                bn_name = block_name + str(l+1)

                conv_weight_name = conv_name + '.weight'
                all_honey_conv_weight.append(conv_weight_name)
                oriweight = oristate_dict[conv_weight_name]
                curweight = state_dict[name_base+conv_weight_name]
                orifilter_num = oriweight.size(0)
                currentfilter_num = curweight.size(0)

                if orifilter_num != currentfilter_num:
                    print('loading rank from: ' + prefix + str(conv_cnt) + subfix)
                    rank = np.load(prefix + str(conv_cnt) + subfix)
                    select_index = np.argsort(rank)[orifilter_num - currentfilter_num:]  # preserved filter id
                    select_index.sort()

                    if (l==6 or (l==0 and layer_cnt!=1) or (l==3 and layer_cnt==1)) and last_select_index is not None:
                        for index_i, i in enumerate(select_index):
                            for index_j, j in enumerate(last_select_index):
                                state_dict[name_base+conv_weight_name][index_i][index_j] = \
                                    oristate_dict[conv_weight_name][i][j]
                            for bn_part in bn_part_name:
                                state_dict[name_base + bn_name + bn_part][index_i] = \
                                    oristate_dict[bn_name + bn_part][i]
                    else:
                        for index_i, i in enumerate(select_index):
                            state_dict[name_base+conv_weight_name][index_i] = \
                                oristate_dict[conv_weight_name][i]
                            for bn_part in bn_part_name:
                                state_dict[name_base + bn_name + bn_part][index_i] = \
                                    oristate_dict[bn_name + bn_part][i]

                    last_select_index = select_index

                elif  (l==6 or (l==0 and layer_cnt!=1) or (l==3 and layer_cnt==1)) and last_select_index is not None:
                    for index_i in range(orifilter_num):
                        for index_j, j in enumerate(last_select_index):
                            state_dict[name_base+conv_weight_name][index_i][index_j] = \
                                oristate_dict[conv_weight_name][index_i][j]
                    for bn_part in bn_part_name:
                        state_dict[name_base + bn_name + bn_part] = \
                            oristate_dict[bn_name + bn_part]
                    last_select_index = None

                else:
                    state_dict[name_base+conv_weight_name] = oriweight
                    for bn_part in bn_part_name:
                        state_dict[name_base + bn_name + bn_part] = \
                            oristate_dict[bn_name + bn_part]
                    last_select_index = None

                state_dict[name_base + bn_name + '.num_batches_tracked'] = oristate_dict[bn_name + '.num_batches_tracked']

            layer_cnt+=1

    for name, module in model.named_modules():
        name = name.replace('module.', '')
        if isinstance(module, nn.Conv2d):
            conv_name = name + '.weight'
            bn_name = list(name[:])
            bn_name[-1] = str(int(name[-1])+1)
            bn_name = ''.join(bn_name)
            if conv_name not in all_honey_conv_weight:
                state_dict[name_base+conv_name] = oristate_dict[conv_name]
                for bn_part in bn_part_name:
                    state_dict[name_base + bn_name + bn_part] = \
                        oristate_dict[bn_name + bn_part]
                state_dict[name_base + bn_name + '.num_batches_tracked'] = oristate_dict[bn_name + '.num_batches_tracked']

        elif isinstance(module, nn.Linear):
            state_dict[name_base+name + '.weight'] = oristate_dict[name + '.weight']
            state_dict[name_base+name + '.bias'] = oristate_dict[name + '.bias']

    model.load_state_dict(state_dict,strict=False)
    print("finish pruning")
    
def adjust_learning_rate(optimizer, epoch, step, len_iter):

    if lr_type == 'step':
        factor = epoch // 30
        if epoch >= 80:
            factor = factor + 1
        lr = learning_rate * (0.1 ** factor)

    elif lr_type == 'cos':  # cos without warm-up
        lr = 0.5 * learning_rate * (1 + math.cos(math.pi * (epoch - 5) / (epochs - 5)))

    elif lr_type == 'exp':
        step = 1
        decay = 0.96
        lr = learning_rate * (decay ** (epoch // step))

    elif lr_type == 'fixed':
        lr = learning_rate
    else:
        raise NotImplementedError

    #Warmup
    if epoch < 5:
        lr = lr * float(1 + step + epoch * len_iter) / (5. * len_iter)

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if step == 0:
        print('learning_rate: ' + str(lr))

def load_resnet_model(model, oristate_dict, layer):
    cfg = {
        56: [9, 9, 9],
        110: [18, 18, 18],
    }

    state_dict = model.state_dict()

    current_cfg = cfg[layer]
    last_select_index = None

    all_conv_weight = []

    prefix = rank_conv_prefix+'rank_conv'
    subfix = ".npy"

    cnt=1
    for layer, num in enumerate(current_cfg):
        layer_name = 'layer' + str(layer + 1) + '.'
        for k in range(num):
            for l in range(2):

                cnt+=1
                cov_id=cnt

                conv_name = layer_name + str(k) + '.conv' + str(l + 1)
                conv_weight_name = conv_name + '.weight'
                all_conv_weight.append(conv_weight_name)
                oriweight = oristate_dict[conv_weight_name]
                curweight =state_dict[name_base+conv_weight_name]
                orifilter_num = oriweight.size(0)
                currentfilter_num = curweight.size(0)

                if orifilter_num != currentfilter_num:
                    print('loading rank from: ' + prefix + str(cov_id) + subfix)
                    rank = np.load(prefix + str(cov_id) + subfix)
                    select_index = np.argsort(rank)[orifilter_num - currentfilter_num:]  # preserved filter id
                    select_index.sort()

                    if last_select_index is not None:
                        for index_i, i in enumerate(select_index):
                            for index_j, j in enumerate(last_select_index):
                                state_dict[name_base+conv_weight_name][index_i][index_j] = \
                                    oristate_dict[conv_weight_name][i][j]
                    else:
                        for index_i, i in enumerate(select_index):
                            state_dict[name_base+conv_weight_name][index_i] = \
                                oristate_dict[conv_weight_name][i]

                    last_select_index = select_index

                elif last_select_index is not None:
                    for index_i in range(orifilter_num):
                        for index_j, j in enumerate(last_select_index):
                            state_dict[name_base+conv_weight_name][index_i][index_j] = \
                                oristate_dict[conv_weight_name][index_i][j]
                    last_select_index = None

                else:
                    state_dict[name_base+conv_weight_name] = oriweight
                    last_select_index = None

    for name, module in model.named_modules():
        name = name.replace('module.', '')

        if isinstance(module, nn.Conv2d):
            conv_name = name + '.weight'
            if 'shortcut' in name:
                continue
            if conv_name not in all_conv_weight:
                state_dict[name_base+conv_name] = oristate_dict[conv_name]

        elif isinstance(module, nn.Linear):
            state_dict[name_base+name + '.weight'] = oristate_dict[name + '.weight']
            state_dict[name_base+name + '.bias'] = oristate_dict[name + '.bias']

    model.load_state_dict(state_dict)

def process_compress_rate(compress_rate):
    if compress_rate:# 处理args.compress_rate
        import re
        cprate_str = compress_rate
        cprate_str_list = cprate_str.split('+')
        pat_cprate = re.compile(r'\d+\.\d*')
        pat_num = re.compile(r'\*\d+')
        cprate = []
        for x in cprate_str_list:
            num = 1
            find_num = re.findall(pat_num, x)
            if find_num:
                assert len(find_num) == 1
                num = int(find_num[0].replace('*', ''))
            find_cprate = re.findall(pat_cprate, x)
            assert len(find_cprate) == 1
            cprate += [float(find_cprate[0])] * num

        compress_rate = cprate
        return compress_rate
    
def load_vgg_model(model, oristate_dict):
    state_dict = model.state_dict()
    last_select_index = None #Conv index selected in the previous layer

    cnt=0
    prefix = rank_conv_prefix+'/rank_conv'
    subfix = ".npy"
    for name, module in model.named_modules():
        name = name.replace('module.', '')

        if isinstance(module, nn.Conv2d):

            cnt+=1
            oriweight = oristate_dict[name + '.weight']
            curweight =state_dict[name_base+name + '.weight']
            orifilter_num = oriweight.size(0)
            currentfilter_num = curweight.size(0)

            if orifilter_num != currentfilter_num:

                cov_id = cnt
                print('loading rank from: ' + prefix + str(cov_id) + subfix)
                rank = np.load(prefix + str(cov_id) + subfix)
                select_index = np.argsort(rank)[orifilter_num-currentfilter_num:]  # preserved filter id
                select_index.sort()

                if last_select_index is not None:
                    for index_i, i in enumerate(select_index):
                        for index_j, j in enumerate(last_select_index):
                            state_dict[name_base+name + '.weight'][index_i][index_j] = \
                                oristate_dict[name + '.weight'][i][j]
                else:
                    for index_i, i in enumerate(select_index):
                       state_dict[name_base+name + '.weight'][index_i] = \
                            oristate_dict[name + '.weight'][i]

                last_select_index = select_index

            elif last_select_index is not None:
                for i in range(orifilter_num):
                    for index_j, j in enumerate(last_select_index):
                        state_dict[name_base+name + '.weight'][i][index_j] = \
                            oristate_dict[name + '.weight'][i][j]
            else:
                state_dict[name_base+name + '.weight'] = oriweight
                last_select_index = None

    model.load_state_dict(state_dict)
    
def load_densenet_model(model, oristate_dict):

    state_dict = model.state_dict()
    last_select_index = [] #Conv index selected in the previous layer

    cnt=0
    prefix = rank_conv_prefix+'/rank_conv'
    subfix = ".npy"
    for name, module in model.named_modules():
        name = name.replace('module.', '')

        if isinstance(module, nn.Conv2d):

            cnt+=1
            cov_id = cnt
            oriweight = oristate_dict[name + '.weight']
            curweight = state_dict[name_base+name + '.weight']
            orifilter_num = oriweight.size(0)
            currentfilter_num = curweight.size(0)

            if orifilter_num != currentfilter_num:
                logger.info('loading rank from: ' + prefix + str(cov_id) + subfix)
                rank = np.load(prefix + str(cov_id) + subfix)
                select_index = list(np.argsort(rank)[orifilter_num-currentfilter_num:])  # preserved filter id
                select_index.sort()

                if last_select_index is not None:
                    for index_i, i in enumerate(select_index):
                        for index_j, j in enumerate(last_select_index):
                            state_dict[name_base+name + '.weight'][index_i][index_j] = \
                                oristate_dict[name + '.weight'][i][j]
                else:
                    for index_i, i in enumerate(select_index):
                        state_dict[name_base+name + '.weight'][index_i] = \
                            oristate_dict[name + '.weight'][i]

            elif last_select_index is not None:
                for i in range(orifilter_num):
                    for index_j, j in enumerate(last_select_index):
                        state_dict[name_base+name + '.weight'][i][index_j] = \
                            oristate_dict[name + '.weight'][i][j]
                select_index = list(range(0, orifilter_num))

            else:
                select_index = list(range(0, orifilter_num))
                state_dict[name_base+name + '.weight'] = oriweight

            if cov_id==1 or cov_id==14 or cov_id==27:
                last_select_index = select_index
            else:
                tmp_select_index = [x+cov_id*12-(cov_id-1)//13*12 for x in select_index]
                last_select_index += tmp_select_index

    model.load_state_dict(state_dict)
    


In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
cudnn.benchmark = True
cudnn.enabled=True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
print("超参数")
CLASSES = 9
lr_type = 'step'
epochs = 300
batch_size=  128
no_graft = False
learning_rate = 0.01
momentum = 0.9
# 0.006
weight_decay = 0.005
label_smooth = 0.1 
lr_decay_step = '150,225'

best_acc = 0
# "mobilenet_v2" "resnet_56" "vgg"
arch = "mobilenet_v2"
# "[0.]+[0.4]*2+[0.5]*9+[0.6]*9+[0.7]*9" “[0.]+[0.15]*2+[0.4]*27”"'[0.]+[0.18]*29'"
# "[0.21]*7+[0.75]*5" "[0.45]*7+[0.75]*5"
#[0.]+[0.2]*2+[0.3]*18+[0.35]*36
compress_rate = "[0.]+[0.3]*7"
# "./data/model/Hrank_preTrain/cifar-10/resnet_56.pt.pt"
pretrain_dir = "./data/model/Hrank_preTrain/teaLevel/mobilenetV2-92-901.t7"
save_dir = "PruneGraft_teaLevel_press2_"+arch
data_dir = "./data/teaLevel/singleLeaf/"
rank_conv_dir = "./data/model/rank_conv/teaLevel/mobileNetV2_limit5/"
print(save_dir)

muti_net = False

超参数
PruneGraft_teaLevel_press2_mobilenet_v2


In [41]:
print("prepare compress_rate")
compress_rate = process_compress_rate(compress_rate)
print('compress_rate:' + str(compress_rate))

prepare compress_rate
compress_rate:[0.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]


In [44]:
net_1 = resnet_110(compress_rate=[0.] * 100,num_classes=9).cuda()

input_image_size=128
input_image = torch.randn(1, 3, input_image_size, input_image_size).cuda()
flops, params = profile(net_1, inputs=(input_image,))
print('Params: %.2f' % (params))
print('Flops: %.2f' % (flops))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.LambdaLayer'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.ResNet'>. Treat it as zero Macs and zero Params.
Params: 1727897.00
Flops: 4079813248.00


In [ ]:
print("=====> Building model")
if arch == "mobilenet_v2":
    net_1 = mobilenet_v2(compress_rate,n_class=CLASSES)
    net_2 = mobilenet_v2(compress_rate,n_class=CLASSES)
    net_1.to(device)
    net_2.to(device)

    rank_conv_prefix = rank_conv_dir
    name_base=''

    origin_model = mobilenet_v2(compress_rate=[0.] * 100,n_class=CLASSES).cuda()
    ckpt = torch.load(pretrain_dir)
    # ckpt = {k:v for k,v in ckpt.items() if "classifier" not in k}
    ckpt = ckpt["net"]

    origin_model.load_state_dict(ckpt)
    oristate_dict = origin_model.state_dict()
    load_mobilenetv2_model(net_1,oristate_dict)
    load_mobilenetv2_model(net_2,oristate_dict)
    
    if muti_net:
        net_3 = mobilenet_v2(compress_rate,n_class=CLASSES).to(device)
        load_mobilenetv2_model(net_3,oristate_dict)
if arch == "resnet_56":
    net_1 = resnet_56(compress_rate,CLASSES)
    net_2 = resnet_56(compress_rate,CLASSES)
    net_1.to(device)
    net_2.to(device)
    rank_conv_prefix = rank_conv_dir
    name_base=''
    
    origin_model = resnet_56(compress_rate=[0.] * 100,num_classes=CLASSES).cuda()
    ckpt = torch.load(pretrain_dir, map_location='cuda:0')
    ckpt = ckpt["net"]
    origin_model.load_state_dict(ckpt)
    
    oristate_dict = origin_model.state_dict()
    load_resnet_model(net_1, oristate_dict, 56)
    print("-------------------")
    load_resnet_model(net_2, oristate_dict, 56)
    if muti_net:
        net_3 = resnet_56(compress_rate,CLASSES).to(device)
        load_resnet_model(net_3,oristate_dict,56)
if arch == "resnet_110":
    net_1 = resnet_56(compress_rate,CLASSES)
    net_2 = resnet_56(compress_rate,CLASSES)
    net_1.to(device)
    net_2.to(device)
    rank_conv_prefix = rank_conv_dir
    name_base=''
    origin_model = resnet_110(compress_rate=[0.] * 100,num_classes=CLASSES).cuda()
    ckpt = torch.load(pretrain_dir, map_location='cuda:0')
    new_state_dict = OrderedDict()
    for k, v in ckpt['net'].items():
        new_state_dict[k.replace('module.', '')] = v
    origin_model.load_state_dict(new_state_dict)
    
    oristate_dict = origin_model.state_dict()
    load_resnet_model(net_1, oristate_dict, 110)
    print("-------------------")
    load_resnet_model(net_2, oristate_dict, 110)
if arch == "vgg":
    net_1 = vgg_16_bn(compress_rate)
    net_2 = vgg_16_bn(compress_rate)
    net_1.to(device)
    net_2.to(device)
    rank_conv_prefix = rank_conv_dir
    name_base=''
    
    origin_model = vgg_16_bn(compress_rate=[0.] * 100).cuda()
    ckpt = torch.load(pretrain_dir, map_location='cuda:0')
    ckpt = ckpt["net"]
    origin_model.load_state_dict(ckpt)
    oristate_dict = origin_model.state_dict()
    
    load_vgg_model(net_1,oristate_dict)
    load_vgg_model(net_2,oristate_dict)
if arch == "densnet_40":
    net_1 = densenet_40(compress_rate=[0.] * 100)
    net_2 = densenet_40(compress_rate=[0.] * 100)
    net_1.to(device)
    net_2.to(device)
    rank_conv_prefix = rank_conv_dir
    name_base=''
    
    origin_model = densenet_40(compress_rate=[0.] * 100).cuda()
    ckpt = torch.load(pretrain_dir, map_location='cuda:0')
    ckpt = ckpt["net"]
    origin_model.load_state_dict(ckpt)
    
    load_densenet_model(net_1, oristate_dict)
    load_densenet_model(net_2, oristate_dict)

In [12]:
class CrossEntropyLabelSmooth(nn.Module):

  def __init__(self, num_classes, epsilon):
    super(CrossEntropyLabelSmooth, self).__init__()
    self.num_classes = num_classes
    self.epsilon = epsilon
    self.logsoftmax = nn.LogSoftmax(dim=1)

  def forward(self, inputs, targets):
    log_probs = self.logsoftmax(inputs)
    targets = torch.zeros_like(log_probs).scatter_(1, targets.unsqueeze(1), 1)
    targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
    loss = (-targets * log_probs).mean(0).sum()
    return loss   


criterion_1 = nn.CrossEntropyLoss()
criterion_1 = criterion_1.cuda()
criterion_smooth_1 = CrossEntropyLabelSmooth(CLASSES, 0.1)
criterion_smooth_1 = criterion_smooth_1.cuda()

criterion_2 = nn.CrossEntropyLoss()
criterion_2 = criterion_2.cuda()
criterion_smooth_2 = CrossEntropyLabelSmooth(CLASSES, 0.1)
criterion_smooth_2 = criterion_smooth_2.cuda()

optimizer_1 = torch.optim.SGD(net_1.parameters(), learning_rate, momentum=momentum, weight_decay=weight_decay)
optimizer_2 = torch.optim.SGD(net_2.parameters(), 0.001, momentum=momentum, weight_decay=weight_decay)
lr_decay_step = list(map(int, lr_decay_step.split(',')))
scheduler_1 = torch.optim.lr_scheduler.MultiStepLR(optimizer_1, milestones=lr_decay_step, gamma=0.1)
scheduler_2 = torch.optim.lr_scheduler.MultiStepLR(optimizer_2, milestones=lr_decay_step, gamma=0.1)

lr_scheduler_1 = torch.optim.lr_scheduler.StepLR(optimizer_1, step_size=60, gamma=0.1)
lr_scheduler_2 = torch.optim.lr_scheduler.StepLR(optimizer_2, step_size=60, gamma=0.1)

if muti_net:
    criterion_3 = nn.CrossEntropyLoss()
    criterion_3 = criterion_1.cuda()
    criterion_smooth_3 = CrossEntropyLabelSmooth(CLASSES, 0.1)
    criterion_smooth_3 = criterion_smooth_1.cuda()
    optimizer_3 = torch.optim.SGD(net_3.parameters(), 0.2, momentum=momentum, weight_decay=0.0006)
    scheduler_3 = torch.optim.lr_scheduler.MultiStepLR(optimizer_3, milestones=lr_decay_step, gamma=0.1)

In [13]:
if not no_graft:
    for epoch in range(300):
        scheduler_1.step()
 #       scheduler_2.step()
        if muti_net: scheduler_3.step()
    ck1 = torch.load("./data/model/PruneGraft_teaLevel_press2_mobilenet_v2/moblie-92.t7")
 #   ck2 = torch.load("./data/model/PruneGraft_teaLevel_press1_vgg/vgg-nograft-92-284.t7")
    net_1.load_state_dict(ck1["net"])
#    net_2.load_state_dict(ck2["net"])
    if muti_net: net_3.load_state_dict(ck["net"])

D:\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [15]:
print('load training data')
data_transform = {
    "train": transforms.Compose([transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])]),
    "val": transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])}

train_dataset = datasets.ImageFolder(root=data_dir+"train",transform=data_transform["train"])
train_num = len(train_dataset)

tea_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in tea_list.items())
json_str = json.dumps(cla_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)

trainloader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size, shuffle=True,
                                           num_workers=0)

validate_dataset = datasets.ImageFolder(root=data_dir + "val",transform=data_transform["val"])
val_num = len(validate_dataset)
testloader = torch.utils.data.DataLoader(validate_dataset,
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=0)

load training data


In [16]:
def entropy(x, n=10):
    x = x.reshape(-1)
    scale = (x.max() - x.min()) / n
    entropy = 0
    for i in range(n):
        p = torch.sum((x >= x.min() + i * scale) * (x < x.min() + (i + 1) * scale), dtype=torch.float) / len(x)
        if p != 0:
            entropy -= p * torch.log(p)
    return float(entropy.cpu())
def l2_norm(weight_torch):
    weight_vec = weight_torch.view(weight_torch.size()[0],-1)
    norm = torch.norm(weight_vec, 2, 1)
    norm_np = norm.cpu().numpy()
    return sum(norm_np)
def Geometric_dist(weight_torch,n=2):
    weight_vec = weight_torch.view(weight_torch.size()[0],-1)
    weight_vec = weight_vec.cpu().numpy()
    similar_matrix = distance.cdist(weight_vec, weight_vec, 'euclidean')
    similar_sum = np.sum(np.abs(similar_matrix), axis=0)
    return similar_sum[similar_large_index]-similar_sum[similar_small_index]
def disEn(weight_torch):
        weight_vec = weight_torch.view(weight_torch.size()[0],-1)
        weight_vec = weight_vec.cpu().numpy()
        similar_matrix = distance.cdist(weight_vec, weight_vec, 'euclidean')
        similar_sum = np.sum(np.abs(similar_matrix), axis=0)
        x = torch.Tensor(similar_sum).cuda()
        n = 5
        scale = (x.max() - x.min()) / n
        entropy = 0
        for i in range(n):
            p = torch.sum((x >= x.min() + i * scale) * (x < x.min() + (i + 1) * scale), dtype=torch.float) / len(x)
            if p != 0:
                entropy -= p * torch.log(p)
        return float(entropy.cpu())
def train(epoch,i,net,optimizer,scheduler,criterion):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    num_iter = len(trainloader)
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
#         adjust_learning_rate(optimizer, epoch, batch_idx, num_iter)
        
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 1000 == 1000 - 1 or 1000 == trainloader.__len__() - 1:
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (
                train_loss / (batch_idx + 1), 100. * correct / total, correct, total))
    scheduler.step()  
def test(epoch,net,i,criterion):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    if acc > best_acc:
        best_acc = acc
        state = {
            'net': net.state_dict(),
            'acc': acc
        }
        torch.save(state, './data/model/'+save_dir+'/best_%d.t7' % (epoch))
    print('Network:%d    epoch:%d    accuracy:%.3f    best:%.3f' % (i, epoch, acc, best_acc))
def grafting(net, epoch,i):
    while True:
        try:
            checkpoint = torch.load('./data/model/'+save_dir+'/ckpt%d_%d.t7' % (i - 1, epoch))['net']
            break
        except:
            time.sleep(10)
    model = collections.OrderedDict()
    w = 1 
    for i, (key, u) in enumerate(net.state_dict().items()):
        if 'conv' in key and 'weight' in key:
            w = round(0.4 / np.pi * np.arctan(500 * (disEn(u) - disEn(checkpoint[key]))) + 0.5, 2)
        model[key] = u * w + checkpoint[key] * (1 - w)
        
    net.load_state_dict(model)
    
# for epoch in range(200):
#     scheduler_1.step()
#     scheduler_2.step()
if muti_net: num = 3
else: num = 2
print(num)

2


In [17]:

if  no_graft:
    for epoch in range(300):
        train(epoch,1,net_1,optimizer_1,scheduler_1,criterion_smooth_1)
        test(epoch,net_1,1,criterion_1)
        state = {
            'net': net_1.state_dict(),
        }
        torch.save(state, './data/model/'+save_dir+'/ckpt%d_%d.t7' % ( 1%num, epoch))
else:
    for epoch in range(300):
        train(epoch,1,net_1,optimizer_1,scheduler_1,criterion_smooth_1)
        test(epoch,net_1,1,criterion_1)
        state = {
            'net': net_1.state_dict(),
        }
        torch.save(state, './data/model/'+save_dir+'/ckpt%d_%d.t7' % ( 1%num, epoch))

        train(epoch,2,net_2,optimizer_2,scheduler_2,criterion_smooth_2)
        test(epoch,net_2,2,criterion_2)
        state = {
            'net': net_2.state_dict(),

        }
        torch.save(state, './data/model/'+save_dir+'/ckpt%d_%d.t7' % ( 2%num, epoch))
        if muti_net:
            train(epoch,3,net_3,optimizer_3,scheduler_3,criterion_smooth_3)
            test(epoch,net_3,3,criterion_3)
            state = {
                'net': net_3.state_dict(),

            }
            torch.save(state, './data/model/'+save_dir+'/ckpt%d_%d.t7' % ( 300%num, epoch))
            grafting(net_3,epoch,3)
        grafting(net_1,epoch,1)
        grafting(net_2,epoch,2)

Network:1    epoch:0    accuracy:91.667    best:91.667
Network:2    epoch:0    accuracy:30.556    best:91.667
Network:1    epoch:1    accuracy:29.630    best:91.667
Network:2    epoch:1    accuracy:39.198    best:91.667
Network:1    epoch:2    accuracy:78.086    best:91.667
Network:2    epoch:2    accuracy:68.210    best:91.667
Network:1    epoch:3    accuracy:78.086    best:91.667
Network:2    epoch:3    accuracy:84.877    best:91.667
Network:1    epoch:4    accuracy:83.642    best:91.667
Network:2    epoch:4    accuracy:80.864    best:91.667
Network:1    epoch:5    accuracy:81.790    best:91.667
Network:2    epoch:5    accuracy:79.321    best:91.667
Network:1    epoch:6    accuracy:72.222    best:91.667
Network:2    epoch:6    accuracy:87.037    best:91.667
Network:1    epoch:7    accuracy:80.556    best:91.667
Network:2    epoch:7    accuracy:85.494    best:91.667
Network:1    epoch:8    accuracy:72.840    best:91.667
Network:2    epoch:8    accuracy:79.938    best:91.667
Network:1 

Network:2    epoch:73    accuracy:89.815    best:91.667
Network:1    epoch:74    accuracy:78.395    best:91.667
Network:2    epoch:74    accuracy:90.741    best:91.667
Network:1    epoch:75    accuracy:79.012    best:91.667
Network:2    epoch:75    accuracy:87.037    best:91.667
Network:1    epoch:76    accuracy:79.630    best:91.667
Network:2    epoch:76    accuracy:90.123    best:91.667
Network:1    epoch:77    accuracy:76.543    best:91.667
Network:2    epoch:77    accuracy:87.346    best:91.667
Network:1    epoch:78    accuracy:73.457    best:91.667
Network:2    epoch:78    accuracy:89.198    best:91.667
Network:1    epoch:79    accuracy:80.556    best:91.667
Network:2    epoch:79    accuracy:88.272    best:91.667
Network:1    epoch:80    accuracy:79.938    best:91.667
Network:2    epoch:80    accuracy:87.654    best:91.667
Network:1    epoch:81    accuracy:79.938    best:91.667
Network:2    epoch:81    accuracy:86.728    best:91.667
Network:1    epoch:82    accuracy:78.395    best

Network:1    epoch:146    accuracy:80.556    best:91.667
Network:2    epoch:146    accuracy:89.506    best:91.667
Network:1    epoch:147    accuracy:77.778    best:91.667
Network:2    epoch:147    accuracy:81.790    best:91.667
Network:1    epoch:148    accuracy:75.926    best:91.667
Network:2    epoch:148    accuracy:89.198    best:91.667
Network:1    epoch:149    accuracy:80.247    best:91.667
Network:2    epoch:149    accuracy:88.889    best:91.667
Network:1    epoch:150    accuracy:79.012    best:91.667
Network:2    epoch:150    accuracy:91.049    best:91.667
Network:1    epoch:151    accuracy:79.938    best:91.667
Network:2    epoch:151    accuracy:89.198    best:91.667
Network:1    epoch:152    accuracy:81.481    best:91.667
Network:2    epoch:152    accuracy:88.580    best:91.667
Network:1    epoch:153    accuracy:81.173    best:91.667
Network:2    epoch:153    accuracy:87.346    best:91.667
Network:1    epoch:154    accuracy:80.864    best:91.667
Network:2    epoch:154    accur

Network:1    epoch:218    accuracy:75.617    best:91.667
Network:2    epoch:218    accuracy:90.123    best:91.667
Network:1    epoch:219    accuracy:79.630    best:91.667
Network:2    epoch:219    accuracy:87.346    best:91.667
Network:1    epoch:220    accuracy:80.556    best:91.667
Network:2    epoch:220    accuracy:88.889    best:91.667
Network:1    epoch:221    accuracy:79.012    best:91.667
Network:2    epoch:221    accuracy:89.506    best:91.667
Network:1    epoch:222    accuracy:81.481    best:91.667
Network:2    epoch:222    accuracy:87.654    best:91.667
Network:1    epoch:223    accuracy:80.556    best:91.667
Network:2    epoch:223    accuracy:88.580    best:91.667
Network:1    epoch:224    accuracy:78.704    best:91.667
Network:2    epoch:224    accuracy:89.506    best:91.667
Network:1    epoch:225    accuracy:71.914    best:91.667
Network:2    epoch:225    accuracy:88.272    best:91.667
Network:1    epoch:226    accuracy:85.185    best:91.667
Network:2    epoch:226    accur

Network:1    epoch:290    accuracy:79.938    best:91.667
Network:2    epoch:290    accuracy:88.580    best:91.667
Network:1    epoch:291    accuracy:81.173    best:91.667
Network:2    epoch:291    accuracy:89.506    best:91.667
Network:1    epoch:292    accuracy:80.864    best:91.667
Network:2    epoch:292    accuracy:89.506    best:91.667
Network:1    epoch:293    accuracy:79.630    best:91.667
Network:2    epoch:293    accuracy:89.198    best:91.667
Network:1    epoch:294    accuracy:75.617    best:91.667
Network:2    epoch:294    accuracy:91.667    best:91.667
Network:1    epoch:295    accuracy:80.864    best:91.667
Network:2    epoch:295    accuracy:87.963    best:91.667
Network:1    epoch:296    accuracy:81.173    best:91.667
Network:2    epoch:296    accuracy:87.963    best:91.667
Network:1    epoch:297    accuracy:80.864    best:91.667
Network:2    epoch:297    accuracy:88.580    best:91.667
Network:1    epoch:298    accuracy:82.099    best:91.667
Network:2    epoch:298    accur

In [18]:
print(net_1.state_dict().keys())

odict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.running_var', 'features.0.1.num_batches_tracked', 'features.1.conv.0.weight', 'features.1.conv.1.weight', 'features.1.conv.1.bias', 'features.1.conv.1.running_mean', 'features.1.conv.1.running_var', 'features.1.conv.1.num_batches_tracked', 'features.1.conv.3.weight', 'features.1.conv.4.weight', 'features.1.conv.4.bias', 'features.1.conv.4.running_mean', 'features.1.conv.4.running_var', 'features.1.conv.4.num_batches_tracked', 'features.2.conv.0.weight', 'features.2.conv.1.weight', 'features.2.conv.1.bias', 'features.2.conv.1.running_mean', 'features.2.conv.1.running_var', 'features.2.conv.1.num_batches_tracked', 'features.2.conv.3.weight', 'features.2.conv.4.weight', 'features.2.conv.4.bias', 'features.2.conv.4.running_mean', 'features.2.conv.4.running_var', 'features.2.conv.4.num_batches_tracked', 'features.2.conv.6.weight', 'features.2.conv.7.weight', 'features.2.